In [2]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=248
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792929/scfs'
TARGET_SPECIES='N'
####################################################


os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()




def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()





av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    
    
    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")


    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(os.getcwd()))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s=\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()




Begin on Mon Feb 19 14:43:29 2018

Working dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792929/scfs


TEMP:            248
TARGET_DIR:      /Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792929/scfs
TARGET_SPECIES:  N


There are 901 efg files in this directory.

901 efg files opened without error.

The indices of N atoms are:
1 3 4 6 23 25 26 28 37 39 40 42 51 53 54 56 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
skipped:847
skipped:848
skipped:849
skipped:850
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipp

Fetching the angles in file efg.{}.out
skipped:847
skipped:848
skipped:849
skipped:850
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipped:892
skipped:893
skipped:894
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	25
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792929/scfs
<tx>:	 -0.000376990042971 	<ty>:	 -0.00793028341209 
 <tx^2>	: 0.0021229419188 	<ty^2>: 0.00418426119011 

cq0=	-6.1074
eta0=	0.47136
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792929/scfs
cwd:

Fetching the angles in file efg.{}.out
skipped:847
skipped:848
skipped:849
skipped:850
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipped:892
skipped:893
skipped:894
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	42
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792929/scfs
<tx>:	 -0.0110034998762 	<ty>:	 0.00190898034536 
 <tx^2>	: 0.00407504079819 	<ty^2>: 0.00267655625703 

cq0=	-6.2174
eta0=	0.50856
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792929/scfs
cwd:	/